## Expanding function to be used to expand theorems to help comprehension

In [1]:
from adapt_utils import replace_symbols

from tree_parser import *

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

import copy

import time

from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph

In [2]:
%%time

text = file_contents()
database = meta_math_database(text,n=16000)

included 5555695 tokens from set.mm
proposition: 16000Wall time: 16min 1s


In [6]:
prop = database.propositions["opoeALTV"]

KeyError: 'opoeALTV'

In [8]:
len(database.propositions_list)

16000

In [61]:
next_i = 5000

In [65]:
for p in database.propositions_list:
    if "Even" in replace_symbols(" ".join(p.statement)):
        print(p.label)
        print(replace_symbols(" ".join(p.statement)))
        break

In [72]:
next_i += 50
print(next_i)

[(p.label, replace_symbols(" ".join(p.statement))) for p in database.propositions_list[next_i:next_i+50]]

5400


[('dmresv', '|- dom ( A |` V ) = dom A'),
 ('rnresv', '|- ran ( A |` V ) = ran A'),
 ('dfrn4', '|- ran A = ( A " V )'),
 ('csbrn', '|- ⦋ A / x ⦌ ran B = ran ⦋ A / x ⦌ B'),
 ('rescnvcnv', "|- ( `' `' A |` B ) = ( A |` B )"),
 ('cnvcnvres', "|- `' `' ( A |` B ) = ( `' `' A |` B )"),
 ('imacnvcnv', '|- ( `\' `\' A " B ) = ( A " B )'),
 ('dmsnn0', '|- ( A ∈ ( V × V ) ↔ dom { A } ≠ ∅ )'),
 ('rnsnn0', '|- ( A ∈ ( V × V ) ↔ ran { A } ≠ ∅ )'),
 ('dmsn0', '|- dom { ∅ } = ∅'),
 ('cnvsn0', "|- `' { ∅ } = ∅"),
 ('dmsn0el', '|- ( ∅ ∈ A → dom { A } = ∅ )'),
 ('relsn2', '|- ( Rel { A } ↔ dom { A } ≠ ∅ )'),
 ('dmsnopg', '|- ( B ∈ V → dom { ⟨ A , B ⟩ } = { A } )'),
 ('dmsnopss', '|- dom { ⟨ A , B ⟩ } ⊆ { A }'),
 ('dmpropg',
  '|- ( ( B ∈ V ∧ D ∈ W ) → dom { ⟨ A , B ⟩ , ⟨ C , D ⟩ } = { A , C } )'),
 ('dmsnop', '|- dom { ⟨ A , B ⟩ } = { A }'),
 ('dmprop', '|- dom { ⟨ A , B ⟩ , ⟨ C , D ⟩ } = { A , C }'),
 ('dmtpop', '|- dom { ⟨ A , B ⟩ , ⟨ C , D ⟩ , ⟨ E , F ⟩ } = { A , C , E }'),
 ('cnvcnvsn', "|- `' `' {

In [4]:
print_proof_linear_steps("dfss2", database)

[dfss] |- ( A ⊆ B ↔ A = ( A ∩ B ) )
[df-in] |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
[eqeq2i] |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
[abeq2] |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[3bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[pm4.71] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [5]:
#1. Olhar pro passo
#2. Pegar as variaveis F do teorema correspondente ao passo
#3. Preencher dentro dos passos dentro da prova do teorema

In [6]:
print_proof_linear_steps("dfss2", database)

[dfss] |- ( A ⊆ B ↔ A = ( A ∩ B ) )
[df-in] |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
[eqeq2i] |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
[abeq2] |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[3bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[pm4.71] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [9]:
from tree_parser import proof_step

def get_step_replace_dict(step):
    
    e_hyps = [h for h in step.prop.hyps if h.type == 'e']
    assert len(e_hyps) == len(step._prior_entails), print(len(e_hyps), len(step._prior_entails))
    
    repdict = {}
    
    for raw_hyp, rep_hyp in zip(e_hyps, step._prior_entails):
       
        raw_tree = raw_hyp.tree
        rep_tree = rep_hyp.tree

        for pos in raw_tree.breadth_first_position_list():
            raw_subtree = raw_tree.tree_at_position(pos)
            rep_subtree = rep_tree.tree_at_position(pos)
            if raw_subtree.value != rep_subtree.value:
                repdict[raw_subtree.value] = rep_subtree
        
    return repdict

def expand_proof_step(root_step):
    
    expanded_steps = []
    
    if not hasattr(root_step, "replace_dict_list"):
        root_step.replace_dict_list = [get_step_replace_dict(root_step)]
    
    for child_step in root_step.prop.entails_proof_steps:
        #Take the original prop tree
        exp_tree = child_step.prop.tree.copy()
        
        #Take the step transformation that was used to the raw step to the replace step
        replace_dict_list = [get_step_replace_dict(child_step)]
            
        #Get the root step replace dicts so we can iterativelly expand the steps
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree = exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
    return expanded_steps

In [10]:
#Verificar como utilizar a lista previa de replace dicts

prop = database.propositions["impbii"]
root_step = prop.entails_proof_steps[-1]

replace_dict_list = [get_step_replace_dict(root_step)]

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for child_step in root_step.prop.entails_proof_steps:
    new_tree = child_step.tree.copy()#.replace(replace_dict_list[0])
    
    for rep_dict in replace_dict_list:
        new_tree = new_tree.replace(rep_dict)
    
    #.replace(step_rep_dict)
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(new_tree.eval(database, prop)))
    
    
    #print(get_step_replace_dict(child_step), child_step.prop.tree)
    #print(replace_symbols(child_step.tree.copy().replace(step_rep_dict).eval(database, prop)))
   # print()

[mp2] |- ( 𝜑 ↔ 𝜓 )



NameError: name 'exp_step' is not defined

In [11]:
#[p.label for p in database.propositions_list[-300:]]

In [23]:
prop = database.propositions["dfss2"]
root_step = prop.entails_proof_steps[-1]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [bicomi] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )
--- [bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [24]:
root_step = exp_proof[-1]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [sylbb] |- ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) )
--- [sylbbr] |- ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B )
--- [impbii] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [91]:
root_step = database.propositions["impbii"].entails_proof_steps[-1]

In [92]:
step_rep_dict = get_step_replace_dict(root_step)
step_rep_dict

{'wph': <Tree: wi(wph(),wps())>,
 'wps': <Tree: wi(wps(),wph())>,
 'wch': <Tree: wb(wph(),wps())>}

In [93]:
root_step.prop.entails_proof_steps[-2].tree.copy().replace(step_rep_dict).eval(database, database.propositions["impbii"])

'( ( ps -> ph ) -> ( ph <-> ps ) )'

In [94]:
database.propositions["impbii"].entails_proof_steps[-2].tree.eval(database, database.propositions["impbii"])

'( ( ph -> ps ) -> ( ( ps -> ph ) -> ( ph <-> ps ) ) )'

In [95]:
database.propositions["impbii"]

In [96]:
done_steps = []

[mp2] |- ( 𝜑 ↔ 𝜓 )

--- [ax-mp] |- 𝜓
--- [ax-mp] |- 𝜓


In [98]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

2 0


AssertionError: None

In [ ]:
for child_step in root_step.prop.entails_proof_steps:
        #exp_tree = child_step.prop.tree.copy()
        exp_tree = child_step.tree.copy()
        
        #Check if the child step tree has a replace dict list (axioms dont have it I think)
        #replace_dict_list = child_step.replace_dict_list.copy()
        
        replace_dict_list = [get_step_replace_dict(child_step)]
        
        
        #if hasattr(child_step.tree, "replace_dict_list"):
        #    exp_tree.replace_dict_list = child_step.tree.replace_dict_list.copy()
        #else:
            #exp_tree.replace_dict_list = []
            
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)

In [ ]:
def expand_proof_step(root_step):

    #rep_dict = root_step.tree._rep_dict
    
    expanded_steps = []
    
    for child_step in root_step.prop.entails_proof_steps:
        #exp_tree = child_step.prop.tree.copy()
        exp_tree = child_step.tree.copy()
        
        #Check if the child step tree has a replace dict list (axioms dont have it I think)
        #replace_dict_list = child_step.replace_dict_list.copy()
        
        replace_dict_list = [get_step_replace_dict(child_step)]
        
        
        #if hasattr(child_step.tree, "replace_dict_list"):
        #    exp_tree.replace_dict_list = child_step.tree.replace_dict_list.copy()
        #else:
            #exp_tree.replace_dict_list = []
            
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
    return expanded_steps

In [6]:
#database.propositions["mp2"].tree.replace_dict_list

AttributeError: 'Tree' object has no attribute 'replace_dict_list'

In [7]:
database.propositions["impbii"].entails_proof_steps[-1].replace_dict_list

[]

In [9]:
database.propositions["impbii"].entails_proof_steps[-1].tree

<Tree: wb(wph(),wps())>

In [24]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[mp2] |- ( 𝜑 ↔ 𝜒 )

--- [ax-mp] |- ( 𝜒 → 𝜒 )
--- [ax-mp] |- 𝜒


In [25]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))
#print("--- [{}]".format(exp_proof[-1].prop.label), "|-", replace_symbols(exp_proof[-1].tree.eval(database, root_step.context)))

[ax-mp] |- 𝜒



In [26]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

IndexError: list index out of range

In [11]:
done_steps[0].replace_dict_list

[{'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [12]:
done_steps[1].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [13]:
done_steps[2].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wch()>},
 {'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [14]:
done_steps[3].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wch()>},
 {'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [17]:
database.propositions["mp2"].entails_proof_steps[-1].replace_dict_list

[]

In [18]:
database.propositions["mp2"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis mp2.1: |- |-ph>,
 <e_hypothesis mp2.2: |- |-ps>,
 <e_hypothesis mp2.3: |- |-(ph->(ps->ch))>]